In [11]:
!pip install mido

In [ ]:
# Imports

import os
import csv
import mido
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# Save the model weights in this directory
dir = "model_wts"

if not os.path.exists(dir):
  os.mkdir(dir)

In [23]:
# Functions to generate the sample from the CSV file

def import_target_list_from_csv(filename):

	with open(filename, 'r') as f:
		target_list = list(csv.reader(f))

	target_list = [[int(t) for t in target] for target in target_list]

	return target_list


def generate_target_array(target_list):

	n_midi_notes   = 108
	n_pad_song_end = 20

	# Torch RNN input (batch_first = True): (n_batches, seq_len, n_dimensions)
	target = np.zeros((1, len(target_list) + n_pad_song_end, n_midi_notes))

	for token_number, token in enumerate(target_list):
		for note_number in token:
			target[0, token_number, note_number - 1] = 1

	return target


def generate_observation(target, noise=0.5):

	observation = target + 0.5 * np.random.randn(*target.shape)

	return(observation)


In [27]:
midi_filename = './bwv0539.mid'
csv_filename  = midi_filename.replace('.mid', '.csv')

target_list = import_target_list_from_csv(csv_filename)
target = generate_target_array(target_list)
observation = generate_observation(target, 0.5)

In [ ]:
# Variables

input_size = 108
sequence_length = 64
batch_size = 64
num_layers = 2
hidden_size = 256
learning_rate = 0.001
num_epochs = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size, sequence_length):
        super(SimpleRNN, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, input_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden=None):
        if hidden is None:
          batch_size = x.size(0)
          hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

        out, hidden = self.rnn(x, hidden)
        out = self.fc1(out)
        out = self.sig(out)
        return out, hidden

model = SimpleRNN(input_size, num_layers, hidden_size, sequence_length).to(device=device)

# Load data from csv
midi_filename = './bwv0539.mid'
csv_filename  = midi_filename.replace('.mid', '.csv')
target_list = import_target_list_from_csv(csv_filename)
target = generate_target_array(target_list)
observation = generate_observation(target, 0.5)

# TODO: Use 80% of available data for training, 20% for validation
dataset = torch.tensor(target, dtype=torch.float32), torch.tensor(observation, dtype=torch.float32)
train_dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False)

# Loss and optimizer
criterion  = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    epoch_loss = 0
    for ground_truth, input_data in tqdm(train_dataloader):
        ground_truth = ground_truth.to(device=device)
        input_data = input_data.to(device=device)

        output, hidden = model(input_data)
        loss = criterion(output, ground_truth)
        current_loss = loss

        # Learning
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_dataloader)}")


# Save the weights
weights_file = os.path.join(dir, 'SimpleRNNweights.pth')
torch.save(model.state_dict(), weights_file)
print(f"Simple RNN weights saved to {dir}")

100%|██████████| 1/1 [00:01<00:00,  1.24s/it]


Epoch [1/5], Loss: 0.6966413855552673


100%|██████████| 1/1 [00:01<00:00,  1.02s/it]


Epoch [2/5], Loss: 0.6765092015266418


100%|██████████| 1/1 [00:00<00:00,  1.06it/s]


Epoch [3/5], Loss: 0.6457871794700623


100%|██████████| 1/1 [00:01<00:00,  2.00s/it]


Epoch [4/5], Loss: 0.5768253207206726


100%|██████████| 1/1 [00:02<00:00,  2.18s/it]

Epoch [5/5], Loss: 0.46642059087753296


In [16]:
class GRUModel(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size, sequence_length):
        super(GRUModel, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, input_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden=None):
        if hidden is None:
            batch_size = x.size(0)
            hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

        out, hidden = self.gru(x, hidden)
        out = self.fc1(out)
        out = self.sig(out)
        return out, hidden

model = GRUModel(input_size, num_layers, hidden_size, sequence_length).to(device=device)

# Load data from csv
midi_filename = './bwv0539.mid'
csv_filename  = midi_filename.replace('.mid', '.csv')
target_list = import_target_list_from_csv(csv_filename)
target = generate_target_array(target_list)
observation = generate_observation(target, 0.5)

# Use 80% of available data for training, 20% for validation
dataset = torch.tensor(target, dtype=torch.float32), torch.tensor(observation, dtype=torch.float32)
train_dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False)

# Loss and optimizer
criterion  = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    epoch_loss = 0
    for ground_truth, input_data in tqdm(train_dataloader):
        ground_truth = ground_truth.to(device=device)
        input_data = input_data.to(device=device)

        output, hidden = model(input_data)
        loss = criterion(output, ground_truth)
        current_loss = loss

        # Learning
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_dataloader)}")


# Save the weights
weights_file = os.path.join(dir, 'GRUweights.pth')
torch.save(model.state_dict(), weights_file)
print(f"GRU Model weights saved to {dir}")

100%|██████████| 1/1 [00:03<00:00,  3.23s/it]


Epoch [1/5], Loss: 0.6950598955154419


100%|██████████| 1/1 [00:04<00:00,  4.93s/it]


Epoch [2/5], Loss: 0.682159960269928


100%|██████████| 1/1 [00:02<00:00,  2.97s/it]


Epoch [3/5], Loss: 0.6676759719848633


100%|██████████| 1/1 [00:03<00:00,  3.09s/it]


Epoch [4/5], Loss: 0.6469460129737854


100%|██████████| 1/1 [00:03<00:00,  3.38s/it]

Epoch [5/5], Loss: 0.6128711104393005


In [15]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size, sequence_length):
        super(SimpleLSTM, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, input_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden=None):
        if hidden is None:
          batch_size = x.size(0)
          hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
          c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

        out, hidden = self.lstm(x, (hidden, c0))
        out = self.fc1(out)
        out = self.sig(out)
        return out, hidden

model = SimpleLSTM(input_size, num_layers, hidden_size, sequence_length).to(device=device)

# Load data from csv
midi_filename = './bwv0539.mid'
csv_filename  = midi_filename.replace('.mid', '.csv')
target_list = import_target_list_from_csv(csv_filename)
target = generate_target_array(target_list)
observation = generate_observation(target, 0.5)

# TODO: Use 80% of available data for training, 20% for validation
dataset = torch.tensor(target, dtype=torch.float32), torch.tensor(observation, dtype=torch.float32)
train_dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False)

# Loss and optimizer
criterion  = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    epoch_loss = 0
    for ground_truth, input_data in tqdm(train_dataloader):
        ground_truth = ground_truth.to(device=device)
        input_data = input_data.to(device=device)

        output, hidden = model(input_data)
        loss = criterion(output, ground_truth)
        current_loss = loss

        # Learning
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_dataloader)}")

# Save the weights
weights_file = os.path.join(dir, 'LSTMweights.pth')
torch.save(model.state_dict(), weights_file)
print(f"LSTM Model weights saved to {dir}")

100%|██████████| 1/1 [00:01<00:00,  1.20s/it]


Epoch [1/50], Loss: 0.692531168460846


100%|██████████| 1/1 [00:00<00:00,  1.36it/s]


Epoch [2/50], Loss: 0.6859359741210938


100%|██████████| 1/1 [00:00<00:00,  1.39it/s]


...

Epoch [48/50], Loss: 0.0793856531381607


100%|██████████| 1/1 [00:00<00:00,  1.25it/s]


Epoch [49/50], Loss: 0.07932130992412567


100%|██████████| 1/1 [00:00<00:00,  1.33it/s]

Epoch [50/50], Loss: 0.07925736904144287
